<a href="https://colab.research.google.com/github/gokul68/Content_similarity_alg1/blob/master/GloVe_and_Elementary_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from collections import *

In [ ]:
import torch
import torchtext
import re

In [ ]:
glove=torchtext.vocab.GloVe(name='6B',dim=50)

.vector_cache/glove.6B.zip: 862MB [06:27, 2.22MB/s]                           
100%|█████████▉| 398683/400000 [00:15<00:00, 25680.79it/s]

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem import WordNetLemmatizer
stop=set(stopwords.words('english'));

In [ ]:
lemmatiser=WordNetLemmatizer();

In [ ]:
def cleanup(doc):
  f=doc.lower();
  f1=re.split('\W+',f);
  f2=[];
  for i in f1:
    if (i in stop) or (len(i)<=2):
      continue;
    else:
      j=lemmatiser.lemmatize(i,pos='n');
      j1=lemmatiser.lemmatize(j,pos='v');
      j2=lemmatiser.lemmatize(j1,pos='a');
      j3=lemmatiser.lemmatize(j2,wordnet.ADJ);
      f2.append(j3);
  return f2;

In [ ]:
def Vectoriser(cleandoc):
  def default():
    return None;
  CountVector=defaultdict(default);
  for i in cleandoc:
    if CountVector[i]==None:
      CountVector[i]=1;
    else:
      CountVector[i]+=1;
  return CountVector;

In [ ]:
def OrderedBasis(cleandoc1,cleandoc2):
  C1=Vectoriser(cleandoc1);
  C2=Vectoriser(cleandoc2);
  def default():
    return None;
  basis=defaultdict(default);
  for i in C1:
    basis[i]=True;
  for j in C2:
    basis[j]=True;
  for word in basis:
    if C1[word]==None:
      C1[word]=0;
    if C2[word]==None:
      C2[word]=0;
  return [C1,C2];

In [ ]:
def VectorBasis(cleandoc1,cleandoc2):
  V=OrderedBasis(cleandoc1,cleandoc2);
  c1=V[0];
  c2=V[1];
  v1=deque();
  v2=deque();
  for word in c1:
    v1.append(c1[word]);
    v2.append(c2[word]);
  V1=np.array(list(v1));
  V2=np.array(list(v2));
  return [V1,V2];

In [ ]:
def NaiveClustering(cleandoc,keys):
  V=Vectoriser(cleandoc);
  wordcount={i:1 for i in keys};
  clusters={i:deque([i]) for i in keys};
  mean={i:glove[i] for i in keys};
  for word in V:
    vect=glove[word];
    x=0;
    for i in vect:
      if i.item()==0:
        x+=1;
    if x==len(vect):
      continue;
    else:
      c=0;
      Max=None;
      index=None;
      for m in mean:
        if c==0:
          Max=abs(torch.cosine_similarity(vect.unsqueeze(0),mean[m].unsqueeze(0)).item());
          index=m;
          c+=1;
        else:
          val=abs(torch.cosine_similarity(vect.unsqueeze(0),mean[m].unsqueeze(0)).item());
          if val>Max:
            Max=val;
            index=m;
      clusters[index].append(word);
      k=mean[index];
      k=k*wordcount[index];
      newMean=torch.add(k,vect)
      newMean=newMean/(wordcount[index]+1);
      mean[index]=newMean;
      wordcount[index]+=1; 
  return clusters;

In [ ]:
s="The legal system is made up of civil courts, criminal courts and specialty courts such as family law courts and bankruptcy court. Each court has its own jurisdiction, which refers to the cases that the court is allowed to hear. In some instances, a case can only be heard in one type of court. For example, a bankruptcy case must be heard in a bankruptcy court. In other instances, there may be several potential courts with jurisdiction. For example, a federal criminal court and a state criminal court would each have jurisdiction over a crime that is a federal drug offense but that is also an offense on the state level.";

In [ ]:
key=['court','crime','system','jurisdiction'];

In [ ]:
S=cleanup(s);

In [ ]:
NaiveClustering(S,key)

{'court': deque(['court',
        'legal',
        'civil',
        'court',
        'family',
        'law',
        'bankruptcy',
        'case',
        'hear',
        'federal',
        'state']),
 'crime': deque(['crime', 'criminal', 'crime', 'drug', 'offense']),
 'jurisdiction': deque(['jurisdiction', 'jurisdiction']),
 'system': deque(['system',
        'system',
        'make',
        'specialty',
        'refer',
        'allow',
        'instance',
        'one',
        'type',
        'example',
        'must',
        'may',
        'several',
        'potential',
        'would',
        'also',
        'level'])}

In [ ]:
import random

In [ ]:
def RandomTheme(doc,k,w):
  S=cleanup(doc);
  random.shuffle(S);
  keys=S[0:k];
  Cluster=NaiveClustering(S,keys);
  Theme={};
  for i in Cluster:
    Theme[i]=True;
    c=list(Cluster[i]);
    random.shuffle(c);
    Theme.update({i:True for i in c[0:w]});
  return Theme;

In [ ]:
RandomTheme(s,5,3)

{'case': True,
 'court': True,
 'crime': True,
 'criminal': True,
 'drug': True,
 'family': True,
 'federal': True,
 'hear': True,
 'jurisdiction': True,
 'law': True,
 'make': True,
 'refer': True,
 'state': True}

In [ ]:
def Topics(doc,k,w,n):
  topic={};
  for i in range(n):
    theme=RandomTheme(doc,k,w);
    for i in theme:
      d=topic.get(i,False);
      if d==False:
        topic[i]=1;
      else:
        topic[i]+=1;
  Topics=sorted(topic.items(),key=lambda x:x[1]);
  T=[Topics[-i][0] for i in range(1,k+1)];
  return T;

In [ ]:
Topics(s,5,3,1000)

['court', 'jurisdiction', 'bankruptcy', 'criminal', 'offense']